In [1]:
# Задание на повторение материала предыдущего семестра

In [2]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [3]:
# Генерируем уникальный seed
my_code = "Suminov"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [5]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [6]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [7]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [8]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [9]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [10]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [11]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [12]:
sc_train

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.37037,0.316953,0.092105,0.387097,0.186538,0.898128,Murmansk Oblast
1,0.518519,0.218673,0.184211,0.672811,0.479196,0.436952,Kurgan Oblast
2,0.62963,0.31941,0.223684,0.529954,0.35507,0.439571,Chuvash Republic
3,0.555556,0.238329,0.171053,0.626728,0.35507,0.778709,Sverdlovsk Oblast
4,0.481481,0.29484,0.179825,0.520737,0.261888,0.578892,Tomsk Oblast
...,...,...,...,...,...,...,...
1384,0.444444,0.400491,0.27193,0.419355,0.512587,0.262538,Republic of Kalmykia
1385,0.222222,0.255528,0.083333,0.493088,0.633392,0.661254,Novosibirsk Oblast
1386,0.0,0.77887,0.77193,0.235023,0.902797,0.264895,Chechen Republic
1387,0.407407,0.228501,0.135965,0.599078,0.417133,0.671599,Arkhangelsk Oblast


In [13]:
# Вспоминаем алгоритмы решения задачи регрессии: линейную регрессию и метод k ближайших соседей
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

In [14]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [15]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [16]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.02065334986236788
0.02990470645931038
0.020623728438726768
0.02990470645931038
0.011888088889412833
0.013203724406504351
0.014170259934899454


In [17]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [18]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.012190929609144538


In [19]:
# Вспоминаем алгоритмы решения задачи классификации:
# логистическую регрессию, наивный байесовский классификатор и (снова) метод k ближайших соседей
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga'))
c_models.append(LogisticRegression(penalty='l1', solver='saga'))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

In [20]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [21]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [22]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.2840840187806734
0.11254942241462677
0.06899789075814994
0.07784735350101887
0.06899789075814994
0.03234897388746308
0.033666790355147254
0.03232097385388598
0.30214799987065266
0.23942004721398094
0.18257526649007033


In [23]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model

MultinomialNB()

In [24]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.018833733543114052


In [25]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [26]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.37037,0.316953,0.092105,0.387097,0.186538,0.898128
1,0.518519,0.218673,0.184211,0.672811,0.479196,0.436952
2,0.62963,0.31941,0.223684,0.529954,0.35507,0.439571
3,0.555556,0.238329,0.171053,0.626728,0.35507,0.778709
4,0.481481,0.29484,0.179825,0.520737,0.261888,0.578892
...,...,...,...,...,...,...
458,0.37037,0.299754,0.127193,0.456221,0.601573,0.654184
459,0.703704,0.427518,0.372807,0.479263,0.35,0.616341
460,0.592593,0.253071,0.157895,0.56682,0.358392,0.739426
461,0.0,0.542998,0.258772,0.138249,0.211713,0.757758


In [27]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [28]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.29495467496091055
0.24303588864703382
0.23752585891056785
0.2296991232552769


In [29]:
# Выбираем лучшую модель
i_min = sils.index(min(sils))
best_k_model = k_models[i_min]
print(best_k_model)
print(sils[i_min])



KMeans(n_clusters=50)
0.2296991232552769


In [30]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25)
]

In [31]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[KNeighborsRegressor(n_neighbors=20), Lasso(alpha=0.5), KNeighborsRegressor(), Lasso()]


In [32]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv")
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,"468,0","0,0","24,9","31,0","100,0","4,0","0,0","993,8","4,0","-3,0","-0,3","-0,4"
1,"140,0","0,0","27,9","22,0","20,0","1,0","0,0","1004,1","4,0","-2,0","0,6","-1,0"
2,"464,0","0,0","25,6","28,0","20,0","2,0","1,0","1001,2","4,0","-7,0","-1,8","-0,3"
3,"467,0","0,0","26,7","26,0","100,0","3,0","0,0","992,3","4,0","-1,0","1,2","-1,3"
4,"138,0","0,0","29,1","22,0","20,0","1,0","0,0","1001,3","4,0","-2,0","2,5","-1,3"
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,"474,0","0,1","6,4","95,0","4,0","1,0","3,0","999,9","4,0","-1,0","5,9","1,5"
1435,"65,0","0,0","6,4","93,0","10,0","4,0","3,0","983,9","4,0","-1,0","5,2","3,1"
1436,"159,0","0,1","8,5","83,0","10,0","2,0","1,0","969,7","4,0","1,0","1,4","1,9"
1437,"99,0","0,1","7,5","91,0","10,0","3,0","3,0","995,4","4,0","-4,0","2,7","1,7"


In [33]:
data['water_level'] = data['water_level'].str.replace(',','.')
data['precipitation'] = data['precipitation'].str.replace(',','.')
data['temperature'] = data['temperature'].str.replace(',','.')
data['humidity'] = data['humidity'].str.replace(',','.')
data['visibility'] = data['visibility'].str.replace(',','.')
data['wind'] = data['wind'].str.replace(',','.')
data['weather'] = data['weather'].str.replace(',','.')
data['pressure'] = data['pressure'].str.replace(',','.')
data['fire'] = data['fire'].str.replace(',','.')                       
data['wl_changer'] = data['wl_change'].str.replace(',','.')                       
data['temp_change'] = data['temp_change'].str.replace(',','.')                      
data['pressure_change'] = data['pressure_change'].str.replace(',','.')                         

In [34]:
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change,wl_changer
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,"-3,0",-0.3,-0.4,-3.0
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,"-2,0",0.6,-1.0,-2.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,"-7,0",-1.8,-0.3,-7.0
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,"-1,0",1.2,-1.3,-1.0
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,"-2,0",2.5,-1.3,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,"-1,0",5.9,1.5,-1.0
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,"-1,0",5.2,3.1,-1.0
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,"1,0",1.4,1.9,1.0
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,"-4,0",2.7,1.7,-4.0


In [35]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = ['water_level']
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['temp_change', 'precipitation', 'humidity', 'pressure', 'visibility']


In [36]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.


In [37]:
all_lb =  x_labels + y_label

In [38]:

data=data[list(all_lb)]
data

,temp_change,precipitation,humidity,pressure,visibility,water_level
0,-0.3,0.0,31.0,993.8,100.0,468.0
1,0.6,0.0,22.0,1004.1,20.0,140.0
2,-1.8,0.0,28.0,1001.2,20.0,464.0
3,1.2,0.0,26.0,992.3,100.0,467.0
4,2.5,0.0,22.0,1001.3,20.0,138.0
...,...,...,...,...,...,...
1434,5.9,0.1,95.0,999.9,4.0,474.0
1435,5.2,0.0,93.0,983.9,10.0,65.0
1436,1.4,0.1,83.0,969.7,10.0,159.0
1437,2.7,0.1,91.0,995.4,10.0,99.0


In [39]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [40]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [41]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = all_lb

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['temp_change', 'precipitation', 'humidity',
                                  'pressure', 'visibility', 'water_level'])])

In [42]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [43]:
# Устанавливаем названия столбцов
column_names = all_lb
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [44]:
sc_train

,temp_change,precipitation,humidity,pressure,visibility,water_level
0,0.379182,0.157895,0.875000,0.719457,0.20,0.255444
1,0.550186,0.000000,0.590909,0.842383,0.20,0.263819
2,0.654275,0.000000,0.215909,0.718703,0.10,0.598827
3,0.591078,0.000000,0.454545,0.741327,0.20,0.304858
4,0.382900,0.000000,0.772727,0.702112,0.10,0.355946
...,...,...,...,...,...,...
858,0.405204,0.000000,0.886364,0.653092,0.20,0.412898
859,0.368030,0.105263,0.545455,0.761689,0.20,0.355946
860,0.405204,0.000000,0.681818,0.645551,0.10,0.588777
861,0.349442,0.000000,0.954545,0.847662,0.01,0.653266


In [45]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [46]:
# Обучаем модели
for model in my_models1:
    model.fit(x_train, y_train)

In [47]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in my_models1:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.0265606329300377
0.02905825208871899
0.0270963917351819
0.02905825208871899


In [48]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = mses.index(min(mses))
best_my_model = my_models1[i_min]
print('Модель ', best_my_model, ' решает задачу лучше других.') 

Модель  KNeighborsRegressor(n_neighbors=20)  решает задачу лучше других.


In [49]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.06516015052339995


In [50]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25)
]

In [51]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'), LogisticRegression(l1_ratio=0.75, penalty='elasticnet', solver='saga'), MultinomialNB(alpha=0.75), KNeighborsClassifier(), KNeighborsClassifier(n_neighbors=15)]


In [52]:
# Загрузим данные для задачи классификации
data = pd.read_csv("datasets/zoo2.csv")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [53]:
#data.drop(['animal_name'], axis='columns', inplace=True)

In [54]:
#data.head()

In [55]:
data['animal_name'] = pd.get_dummies(data['animal_name'])
data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,0,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3


In [56]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = ['class_type']
x_labels = random.sample(columns[:-1], n_x)
all_lb1 =  x_labels + y_label
print(x_labels)

['hair', 'catsize', 'breathes', 'backbone', 'eggs', 'predator', 'milk', 'toothed']


In [57]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [58]:
data=data[list(all_lb1)]
data.head()

,hair,catsize,breathes,backbone,eggs,predator,milk,toothed,class_type
0,0,1,1,1,1,0,0,0,3
1,0,0,1,1,1,0,0,1,3
2,0,1,1,1,1,1,0,1,3
3,0,0,1,1,1,1,0,1,3
4,0,0,1,1,1,0,0,1,3


In [59]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

9


In [60]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

25 9 9


In [61]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = x_labels#all_lb1

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['hair', 'catsize', 'breathes', 'backbone',
                                  'eggs', 'predator', 'milk', 'toothed'])])

In [62]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [63]:
# Устанавливаем названия столбцов
column_names = all_lb1
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [64]:
sc_train

,hair,catsize,breathes,backbone,eggs,predator,milk,toothed,class_type
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0
2,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,3.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0
7,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0
8,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0
9,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,3.0


In [65]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_label])
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [66]:
# Обучаем модели
for model2 in my_models2:
    model2.fit(x_train, y_train)

In [67]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in my_models2:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.7901234567901235
0.7901234567901235
0.6262626262626262
0.7407407407407409
0.6531986531986531


In [68]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = f1s.index(min(f1s))
best_my_models2 = my_models2[i_min]
print('Модель ', best_my_models2, ' решает задачу лучше других.') 

Модель  MultinomialNB(alpha=0.75)  решает задачу лучше других.


In [69]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_my_models2.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.6902356902356901
